<a href="https://colab.research.google.com/github/patelarth/pegasus-onnews-ads/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#if the notebook is colab 
!pip3 install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install sentencepiece
!pip install transformers
!pip install rouge-score

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     |█████████████▌                  | 834.1 MB 1.6 MB/s eta 0:12:05tcmalloc: large alloc 1147494400 bytes == 0x5627d1b3c000 @  0x7f7d87d0a615 0x5627cfe464cc 0x5627cff2647a 0x5627cfe492ed 0x5627cff3ae1d 0x5627cfebce99 0x5627cfeb79ee 0x5627cfe4abda 0x5627cfebcd00 0x5627cfeb79ee 0x5627cfe4abda 0x5627cfeb9737 0x5627cff3bc66 0x5627cfeb8daf 0x5627cff3bc66 0x5627cfeb8daf 0x5627cff3bc66 0x5627cfeb8daf 0x5627cfe4b039 0x5627cfe8e409 0x5627cfe49c52 0x5627cfebcc25 0x5627cfeb79ee 0x5627cfe4abda 0x5627cfeb9737 0x5627cfeb79ee 0x5627cfe4abda 0x5627cfeb8915 0x5627cfe4aafa 0x5627cfeb8c0d 0x5627cfeb79ee
     |█████████████████               | 1055.7 MB 1.5 MB/s eta 0:10:32tcmalloc: large alloc 1434370048 bytes == 0x562816192000 @  0x7f7d87d0a615 0x5627cfe464cc 0x5627cff2647a 0x5627cfe492ed 0x5627cff3ae1d 0x5627cfebce99 0x5627cfeb79ee 0x5627cfe4abda 0x5627cfebcd00 0x5627cfeb79ee 0x5627cfe4abda 0x5627cfeb9737 0x5627cff3bc66 0x5627c

In [5]:
!pip install datasets


     |████████████████████████████████| 290 kB 7.7 MB/s 
     |████████████████████████████████| 243 kB 68.7 MB/s 
     |████████████████████████████████| 125 kB 51.0 MB/s 
     |████████████████████████████████| 1.1 MB 39.8 MB/s 
     |████████████████████████████████| 192 kB 56.5 MB/s 
     |████████████████████████████████| 160 kB 69.7 MB/s 
     |████████████████████████████████| 271 kB 57.4 MB/s 


In [8]:
from datasets import load_dataset
data = load_dataset("cnn_dailymail",'3.0.0')

  0%|          | 0/5 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/376M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/572k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/661k [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
data


DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [17]:
random = [3401,2451,3001,5015] # random ID to get the random text data
test_data_sum = [] # summary
test_data_doc = [] # artical 
for id in random:
  test_data_doc.append(data['test'][id]['article'])
  test_data_sum.append(data['test'][id]['highlights'])

In [20]:
test_data_sum

['The proposal has been put forward by experts at NHS Health Scotland .\nThey say more must be done to tackle alcohol problem with young people .\nThe move would apply in pubs, clubs, supermarkets and off-licences .',
 'Arsenal earn 1-0 Premier League victory against Burnley at Turf Moor .\nAaron Ramsey gives Gunners an early lead with 12th minute strike .\nGunners close to gap to four points behind Premier League leaders Chelsea .\nBurnley remain in the bottom three with fixtures against fellow strugglers Leicester, Hull and Aston Villa to come .\nCLICK HERE for full player ratings as Francis Coquelin shines for the Gunners .',
 'Aunt and cousins of Waheed Ahmed were arrested at Manchester Airport .\nAhmed, 21, was held by anti-terror police at Birmingham Airport yesterday .\nStudent is accused of trying to take eight family members into Syria .\nHe was arrested in Turkish border town with family, including four children .',
 "Natural World Safaris offers tourists the opportunity to s

**First we train the model on cnn_daily mail and apply the cnn test data and check the results.**

In [21]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
from rouge_score import rouge_scorer

In [22]:
def model_train_pegasus(datasetname = 'google/pegasus-cnn_dailymail'):
  dataset_name = datasetname 
  # check if cude is availabel if yes then use cuda for processing
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  tokenizer_pegasus = PegasusTokenizer.from_pretrained(dataset_name)
  model = PegasusForConditionalGeneration.from_pretrained(dataset_name).to(device)
  return tokenizer_pegasus,model

tokenizer,model = model_train_pegasus() # trained  tokanizer and model

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [26]:
def model_abstractive_summary_gen(src_text,tokenizer,model):
  batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt") #gap sentence gentartion and encoding 
  translated = model.generate(**batch)
  abstrct_sumtext = tokenizer.batch_decode(translated, skip_special_tokens=True) # decode
  return abstrct_sumtext

In [31]:
def check_the_rouge(summary,predicted):
  r1 = [[],[],[]] # metrix for rouge 1 score index 0 for recall, 1 for precision and 2 for fmeasure 
  r2 = [[],[],[]] # metrix for rouge 2 score index 0 for recall, 1 for precision and 2 for fmeasure
  rL = [[],[],[]] # metrix for rouge L score index 0 for recall, 1 for precision and 2 for fmeasure
  scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)

  for sc in range(len(summary)):
    scores = scorer.score(str(summary[sc]),str(predicted[sc]))
    r1[0].append(scores['rouge1'].recall)
    r1[1].append(scores['rouge1'].precision)
    r1[2].append(scores['rouge1'].fmeasure)
    r2[0].append(scores['rouge2'].recall)
    r2[1].append(scores['rouge2'].precision)
    r2[2].append(scores['rouge2'].fmeasure)
    rL[0].append(scores['rougeL'].recall)
    rL[1].append(scores['rougeL'].precision)
    rL[2].append(scores['rougeL'].fmeasure)

  return r1,r2,rL

In [28]:
predicted_test = []
for artical in test_data_doc:
  predicted_test.append(model_abstractive_summary_gen(artical,tokenizer,model))
predicted_test

[['Move would apply in pubs, clubs, supermarkets and off-licences in Scotland.<n>If introduced it would mean the legal minimum age for buying alcohol would be higher in Scotland than it is in the rest of Britain, where it is 18.'],
 ['Arsenal beat Burnley 1-0 in their Premier League clash at Turf Moor.<n>Aaron Ramsey scored the only goal of the game in the 12th minute.<n>The Gunners have now won eight games in a row in all competitions.'],
 ["Two women aged 47 and 22, and two men aged 24 and 22, held at Manchester Airport in the early hours of this morning.<n>They were among eight relatives stopped at the Turkish border with Syria on April 1 with Waheed Ahmed, the 21-year-old son of Labour councillor Shakil Ahmed.<n>The group is understood to include Waheed's aunt, two of his male cousins, his cousin's wife and two of their children.<n>The four children on the trip aged one, three, eight and 11, have also been flown back to the UK."],
 ["Bespoke tour operators Natural World Safaris is 

In [32]:
#check the rouge score of thetrained model on the cnn_dailymail test dataset 
r1,r2,rL =  check_the_rouge(test_data_doc,predicted_test)

In [41]:
import numpy as np
#average recall of the test 
print('rouge 1 average recall : ' ,np.array(r1[0]).mean())
print('rouge 2 average recall : ' ,np.array(r2[0]).mean())
print('rouge L average recall : ' ,np.array(rL[0]).mean())
print('rouge 1 average precision : ' ,np.array(r1[1]).mean())
print('rouge 2 average precision : ' ,np.array(r2[1]).mean())
print('rouge L average precision : ' ,np.array(rL[1]).mean())
print('rouge 1 average fmeasure : ' ,np.array(r1[2]).mean())
print('rouge 2 average fmeasure : ' ,np.array(r2[2]).mean())
print('rouge L average fmeasure : ' ,np.array(rL[2]).mean())

rouge 1 average recall :  0.0733699244218588
rouge 2 average recall :  0.06382249788191238
rouge L average recall :  0.07074163906516293
rouge 1 average precision :  0.9524768748963
rouge 2 average precision :  0.765856534113707
rouge L average precision :  0.8758762651402024
rouge 1 average fmeasure :  0.13409800840310815
rouge 2 average fmeasure :  0.11634714698776444
rouge L average fmeasure :  0.12905533589606494


**Collect new data and apply the model on that and check rouge score**

In [43]:
links_list = ['https://7news.com.au/politics/world-politics/emmanuel-macron-tells-scott-morrison-trust-was-broken-after-aukus-submarine-deal-c-4358673',
              'https://7news.com.au/sport/afl/afl-eagle-jones-charged-with-drink-driving-c-4359074',
              'https://7news.com.au/politics/gladys-berejiklian/berejiklian-next-to-face-grilling-by-icac-c-4360136',
              'https://7news.com.au/sunrise/on-the-show/covid-19-rapid-antigen-tests-now-available-at-supermarkets-and-pharmacies-c-4382586',
              'https://7news.com.au/news/missing-person/nsw-police-issue-alert-for-kathleen-riethmuller-after-the-lane-cove-woman-mysteriously-vanished-c-4378401',
              'https://7news.com.au/lifestyle/health-wellbeing/new-covid-19-cases-in-nsw-drop-to-low-not-seen-since-july-c-4377068',
              'https://7news.com.au/politics/joyce-details-some-of-nationals-2050-deal-c-4377609',
              'https://7news.com.au/news/vic/kill-the-bill-thousands-of-protesters-take-to-melbourne-streets-over-covid-19-pandemic-laws-c-4372996',
              'https://7news.com.au/entertainment/tv/armorer-on-the-set-of-rust-says-she-has-no-idea-where-deadly-ammunition-that-killed-halyna-hutchins-came-from-c-4371273',
              'https://7news.com.au/news/world/just-two-per-cent-of-elon-musks-wealth-could-solve-world-hunger-according-to-un-food-scarcity-organisation-c-4345980',
              'https://www.abc.net.au/everyday/dating-as-a-single-mum-with-chronic-illness/100518792',
              'https://www.abc.net.au/news/2021-11-01/real-estate-housing-affordability-four-corners/100572780',
              'https://www.abc.net.au/news/2021-11-01/jana-pittman-how-australia-s-golden-girl-continues-to-surprise/100525802',
              'https://www.abc.net.au/news/2021-10-31/meet-the-millions-of-people-who-are-not-employed/100582656',
              'https://www.abc.net.au/news/science/2021-10-30/snakes-venomous-how-to-survive-a-bite/100406488',
              'https://www.abc.net.au/news/2021-10-29/uq-researchers-unbreakable-phone-glass/100578398#:~:text=The%20researchers%20have%20developed%20a,TVs%2C%20computers%20and%20virtual%20reality'
              'https://www.abc.net.au/news/2021-11-01/qld-coronavirus-covid-hospital-vaccinations/100574456',
              'https://www.abc.net.au/news/2021-11-01/quarantine-free-travel-resumes-at-sydney-airport/100584228',
              'https://www.abc.net.au/news/2021-11-01/westpac-share-buyback-asx-rises-head-of-rba-meeting/100584526',
              'https://www.abc.net.au/news/2021-11-01/workers-at-melbourne-mcdonalds-allegedly-denied-paid-rest-breaks/100582146'
              ]
              # links of the some new artical from 7 news and ABC news 

In [44]:
# file scrape.py code 
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import pandas as pd

class scrap:

    src = []
    summary = []
    df = pd.DataFrame()

    def __init__(self):
        ''' constructor method for an class'''
        self.src = []
        self.summary = []


    def preprocess_data(self,text):
        '''it will preprocess the text which contain the number and any other special charaters and return the preprocess text'''
        text = re.sub(r'\s+',' ',text)  # remove the special char. from the text
        text = re.sub(r'\d','',text)
        return text


    def scrap_via_bs4(self,links):
        'input link, it will scrap the data from the link and store in variable'
        for link in links:
            url = link
            html = urlopen(url) # make an connection to the link and apply bs4
            soup = BeautifulSoup(html, 'html.parser')
            text = ""
            title = ""
            for paragraph in soup.find_all('p'):
                text += paragraph.get_text()
                process_text = self.preprocess_data(text)
                process_text = '.'.join(process_text.split('.')) + '.'
            for paragraph in soup.find_all('h1'):
                title += paragraph.get_text()
                process_title = self.preprocess_data(text)
                process_title = '.'.join(process_title.split('.')) + '.'
            self.src.append(text)
            self.summary.append (title)

    def genrate_data_frame(self):
        ''' this function will genrate the data frame of the form the src and summary data.
         first have to genrate scrape data from the url'''
        self.df = pd.DataFrame({'text': self.src,
                                'summary': self.summary})
        return self.df

    def create_csv(self,name):
        ''' this function require file name with and extension .csv (other excel format are accepted) with the file path and create file of the scrapped dataset'''
        self.df.to_csv(name)
        print('File created')

In [46]:
object = scrap()
object.scrap_via_bs4(links_list) # scrape data from the news website
df = object.genrate_data_frame()


In [48]:
df.head(5)

,text,summary
0,French President Emmanuel Macron has told Aust...,Emmanuel Macron tells Scott Morrison ‘trust wa...
1,West Coast AFL forward Jamaine Jones will face...,‘Bitterly disappointed’: Eagles forward to fac...
2,Twenty-eight days after a corruption inquiry f...,Gladys Berejiklian to face ICAC on Friday afte...
3,Australians can now buy COVID-19 rapid tests a...,COVID-19 rapid antigen tests now available at ...
4,A 28-year-old woman has been reported missing ...,NSW Police issue alert for Kathleen Riethmulle...


In [ ]:
from google import drive

In [49]:
#now genrate summarie suing the oretrain model cnn_daily mail using the new data
predicted_new = []
for artical in df.summary:
  predicted_new.append(model_abstractive_summary_gen(artical,tokenizer,model))
predicted_new 

[['Emmanuel Macron tells Scott Morrison ‘trust is broken’ after AUKUS submarine deal.<n>French president tells Morrison he will not seek re-election.'],
 ['Eagles set to face court over drinking driving charges.<n>‘Bitterly disappointed,’ says Eagles coach Andy Reid.<n>‘Bitterly disappointed,’ says Reid.<n>‘Bitterly disappointed,’ says Reid.'],
 ['Gladys Berejiklian to face ICAC on Friday after Daryl Maguire aired details of intimate relationship.<n> Gladys Berejiklian to face ICAC on Friday after Daryl Maguire aired details of intimate relationship.'],
 ['COVID-19 rapid antigen tests now available at supermarkets and pharmacies.<n>CoVID-19 rapid antigen tests now available at supermarkets and pharmacies.<n>CoVID-19 rapid antigen tests now available at supermarkets and pharmacies.'],
 ['NSW Police issue alert for Kathleen Riethmuller.<n>Lane Cove woman mysteriously vanished.<n>Police believe she may have been taken by traffickers.'],
 ['New COVID-19 cases in NSW drop to low not seen si

In [50]:
# check the rouge score for new data 
r1,r2,rL =  check_the_rouge(df.summary,predicted_new)

In [51]:
#average recall of the test on new data
print('rouge 1 average recall : ' ,np.array(r1[0]).mean())
print('rouge 2 average recall : ' ,np.array(r2[0]).mean())
print('rouge L average recall : ' ,np.array(rL[0]).mean())
print('rouge 1 average precision : ' ,np.array(r1[1]).mean())
print('rouge 2 average precision : ' ,np.array(r2[1]).mean())
print('rouge L average precision : ' ,np.array(rL[1]).mean())
print('rouge 1 average fmeasure : ' ,np.array(r1[2]).mean())
print('rouge 2 average fmeasure : ' ,np.array(r2[2]).mean())
print('rouge L average fmeasure : ' ,np.array(rL[2]).mean())

rouge 1 average recall :  0.9477443609022558
rouge 2 average recall :  0.9043500359289832
rouge L average recall :  0.9302005012531327
rouge 1 average precision :  0.4153292311479277
rouge 2 average precision :  0.37881958310131747
rouge L average precision :  0.40767119717410427
rouge 1 average fmeasure :  0.5701204916587999
rouge 2 average fmeasure :  0.5265229551001788
rouge L average fmeasure :  0.5595346303997439
